<a href="https://colab.research.google.com/github/mscitpract/nlp/blob/main/Normalized_Web_Distance_and_Word_Similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install textdistance

In [3]:
import numpy as np
import re
import textdistance
# we will need scikit-learn>=0.21
import sklearn  # pip install sklearn
from sklearn.cluster import AgglomerativeClustering

texts = [
    'Reliance supermarket', 'Reliance hypermarket', 'Reliance', 'Reliance',
    'Reliance downtown', 'Reliance market', 'Mumbai', 'Mumbai Hyper',
    'Mumbai dxb', 'mumbai airport', 'k.m trading', 'KM Trading',
    'KM trade', 'K.M. Trading', 'KM.Trading'
]

def normalize(text):
    """ Keep only lower-cased text and numbers"""
    return re.sub('[^a-z0-9]+', ' ', text.lower())

def group_texts(texts, threshold=0.4):
    """ Replace each text with the representative of its cluster"""
    normalized_texts = np.array([normalize(text) for text in texts])
    distances = 1 - np.array([
        [textdistance.jaro_winkler(one, another) for one in normalized_texts]
        for another in normalized_texts
    ])
    clustering = AgglomerativeClustering(
        distance_threshold=threshold,  # this parameter needs to be tuned carefully
        affinity="precomputed", linkage="complete", n_clusters=None
    ).fit(distances)

    # clustering = AgglomerativeClustering(
    #     distance_threshold=threshold,  # this parameter needs to be tuned carefully
    #     metric="precomputed", linkage="complete", n_clusters=None
    # ).fit(distances)   ## Use this if above line is not working

    centers = dict()
    for cluster_id in set(clustering.labels_):
        index = clustering.labels_ == cluster_id
        centrality = distances[:, index][index].sum(axis=1)
        centers[cluster_id] = normalized_texts[index][centrality.argmin()]

    return [centers[i] for i in clustering.labels_]

print(group_texts(texts))

['reliance', 'reliance', 'reliance', 'reliance', 'reliance', 'reliance', 'mumbai', 'mumbai', 'mumbai', 'mumbai', 'km trading', 'km trading', 'km trading', 'km trading', 'km trading']


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_agglomerative.py:1006: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(
